<a href="https://colab.research.google.com/github/Blink-SEO/googleads_api_wrapper/blob/main/GoogleAdsWrapperDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

We clone the googleads_api_wrapper package from the Blink-SEO GitHub repo, and also install the google-ads api python client library. 

If this raises an error, try restarting the runtime and try again. I seem to be getting errors using `pip install google-ads` from within Google Colab if it's already been installed once -- a fresh 

email: joshua@blinkseo.co.uk
github repo: https://github.com/Blink-SEO/googleads_api_wrapper

In [ ]:
!git clone https://github.com/Blink-SEO/googleads_api_wrapper.git
!pip install google-ads

In [22]:
from google.colab import files
from google.ads.googleads.client import GoogleAdsClient
from googleads_api_wrapper.clientwrapper import KeywordPlanService, KeywordPlanIdeaService

# Creating a GoogleAds client

Google provides a python client library for its API. Contructing the client object requires some credentials stored in a yaml file. If you do not already have a Google Ads Test Account or haven't obtained these required credentials, there is an excellent guide at https://www.cahoover.com/blog/marketing-analytics/getting-started-with-the-google-ads-api-using-python/ where the process is explained step-by-step. You must already have a regular Google Ads account and a Google Cloud Platform account, as well as being able to use pythion and some command line tools. 

During the process you will also obtain a customer id, one for each test account. You will need to copy-and-paste this into this notebook.

In [23]:
CUSTOMER_ID = "2634951801"  # One of Blink-SEO's customer ids, paste your own here (wrapped in "" and without hyphens)  

In [24]:
# file upload prompt. please upload a yaml file named "google-ads.yaml"
files.upload()

Saving google-ads.yaml to google-ads (2).yaml


{'google-ads.yaml': b'# blink-test-account | test manager account | 9530168619\n# testclient | regular account withing blink-test-account | 2634951801\n# Blink PPC | manager account | 9530779737\n\n# Developer token\n##########################################################################################\n# A developer token is required when making requests to the Google Ads API regardless of #\n# whether you\'re using the OAuth2 or Service Account configurations. To obtain a         #\n# developer token see:                                                                   #\n# https://developers.google.com/google-ads/api/docs/first-call/dev-token                 #\n##########################################################################################\ndeveloper_token: jRsu5yhtQGM5UO594zg3lA\n\n# Use proto plus\n##########################################################################################\n# This parameter specifies whether the ads_client library should return proto

In [25]:
# read the yaml file and use the credentials to create a GoogleAdsClient object
with open("google-ads.yaml", "r") as f:
    googleads_yaml_string = f.read()
ADS_CLIENT = GoogleAdsClient.load_from_string(yaml_str=googleads_yaml_string)

# Using the wrapper

Now we have our GoogleAdsClient object and our customer id we can pass these into our wrapper classes, KeywordPlanService and KeywordPlanIdeaService. 

We can also pass into these classes a default location and languge, so that we don't have to pass these in on every request. The location can be specified as a 3-letter iso code (e.g. GBR) or a number corresponding to a more specific region as defined by Ggoogle Ads in this handy csv file: https://developers.google.com/google-ads/api/reference/data/geotargets

If we don't specify these, location_codes defaults to "GBR" and language_id defaults to "1000" (i.e. English).

The KeywordPlanIdeaService class also takes an optional `site_url` parameter, which is used to generate keyword suggestions.

In [44]:
kps = KeywordPlanService(googleads_client=ADS_CLIENT, customer_id=CUSTOMER_ID)
kpis = KeywordPlanIdeaService(googleads_client=ADS_CLIENT, customer_id=CUSTOMER_ID, site_url="https://www.blinkseo.co.uk/")

In [45]:
# The code below asks for keyword metrics for keywords "hot fudge", "ice cream" and "chocolate pudding"
# note we haven't passed in a location_codes parameter, so the metrics will be for the location codes 
# specified when we created the `kps` object.
# It looks like volume is on the increase for "ice cream" (time of writing: April 2022)
kps.get_keyword_metrics(keywords=["hot fudge", "ice cream", "chocolate pudding"])  

,date_obtained,query,avg_searches,competition,competition_index,low_top_of_page_bid_micros,high_top_of_page_bid_micros,volume_trend,volume_trend_coef,latest_volume
0,2022-04-21,hot fudge,480,LOW,20,503057,1169554,"[390, 1600, 390, 390, 390, 320, 480, 390, 390,...",1.049285e-01,590
1,2022-04-21,ice cream,301000,MEDIUM,36,694675,2619486,"[165000, 673000, 368000, 450000, 550000, 45000...",2.262268e-01,246000
2,2022-04-21,chocolate pudding,5400,MEDIUM,66,584853,1535630,"[4400, 5400, 3600, 3600, 4400, 4400, 5400, 660...",-3.992370e-17,5400


In [ ]:
# The code below will generate some keyword ideas for the site.
# If we have already specified a site_url when creating the KeywordPlanIdeaService object, we can just specify an extension here.
# Passing url="/" or url="" will generate ideas for the home page.
kpis.generate_keyword_ideas(url="/case-studies/")

In [ ]:
# The code below will generate some keyword ideas for the site based on both the site homepage and also the phrases "digital" and "SEO"
# It's not great tbh. 
kpis.generate_keyword_ideas(url="/", phrases=["digital", "SEO"])